### Constants

In [1]:
gDaysInYear = 365.0

### Time Utils

In [2]:
import calendar
import holidays
import numpy as np
import datetime as dt
from dateutil.relativedelta import relativedelta
from enum import Enum

In [3]:
# References:
# https://developers.opengamma.com/quantitative-research/Interest-Rate-Instruments-and-Market-Conventions.pdf
# https://en.wikipedia.org/wiki/Day_count_convention
# http://www.fairmat.com/documentation/usermanual/topics/download/mediawiki/index.php/Day_Count_Conventions.htm
# http://www.eclipsesoftware.biz/DayCountConventions.html
# https://github.com/domokane/FinancePy/blob/master/financepy

In [4]:
class FrequencyTypes(Enum):
    ZERO = -1
    SIMPLE = 0
    ANNUAL = 1
    SEMI_ANNUAL = 2
    TRI_ANNUAL = 3
    QUARTERLY = 4
    MONTHLY = 12
    CONTINUOUS = 99

In [5]:
def get_annual_frequency(frequency_type: FrequencyTypes):
    """Returns the number of payments in a year given a frequency_type."""
    frequency_dict = {FrequencyTypes.CONTINUOUS: -1.0,
                      FrequencyTypes.ZERO: 1.0,
                      FrequencyTypes.ANNUAL: 1.0,
                      FrequencyTypes.SEMI_ANNUAL: 2.0,
                      FrequencyTypes.TRI_ANNUAL: 3.0,
                      FrequencyTypes.QUARTERLY: 4.0,
                      FrequencyTypes.MONTHLY: 12.0}
    return frequency_dict[frequency_type]

In [6]:
class DayCountBasisTypes(Enum):
    ZERO = 0
    THIRTY_360_BOND = 1
    THIRTY_E_360 = 2
    THIRTY_E_360_ISDA = 3
    THIRTY_E_PLUS_360 = 4
    ACT_ACT_ISDA = 5
    ACT_ACT_ICMA = 6
    ACT_365F = 7
    ACT_360 = 8
    ACT_365L = 9
    SIMPLE = 10

In [7]:
def get_dmy(date: dt.date):
    """Returns day, month, and year of a dt.date object.""" 
    day = date.day
    month = date.month
    year = date.year
    return day, month, year

In [8]:
def is_eom(date: dt.date):
    """Checks if a dt.date object is the end of a month date."""
    return date.day == calendar.monthrange(date.year, date.month)[1]

In [9]:
def is_leap_year(date: dt.date):
    """Checks is a dt.date is in a leap year."""
    return date.year % 4 == 0 and (date.year % 100 != 0 or date.year % 400 == 0)

In [10]:
# TODO: Check if DayCount class works properly

class DayCount(object):
    
    def __init__(self, day_count_basis_type: DayCountBasisTypes):
        
        self.day_count_basis_type = day_count_basis_type
    
    
    def year_fraction(self,
                  coupon_period_start: dt.date,
                  settlement_date: dt.date,
                  coupon_period_end: dt.date = None,
                  frequency_type: FrequencyTypes = FrequencyTypes.ANNUAL,
                  termination_date_flag: bool = False): # Flag to check if settlement_date is a termination date 
        
        day_count_basis_type = self.day_count_basis_type
        start_day, start_month, start_year = get_dmy(coupon_period_start)
        end_day, end_month, end_year = get_dmy(settlement_date)
        
        if day_count_basis_type == DayCountBasisTypes.THIRTY_360_BOND:
            if start_day == 31: start_day = 30
            if end_day == 31 and start_day == 30: end_day = 30
            numerator = 360 * (end_year - start_year) + 30 * (end_month - start_month) + (end_day - start_day)
            denominator = 360
            acc_factor = numerator / denominator
            return acc_factor, numerator, denominator
            
        elif day_count_basis_type == DayCountBasisTypes.THIRTY_E_360:
            if start_day == 31: start_day = 30
            if end_day == 31: end_day = 30
            numerator = 360 * (end_year - start_year) + 30 * (end_month - start_month) + (end_day - start_day)
            denominator = 360
            acc_factor = numerator / denominator
            return acc_factor, numerator, denominator
        
        elif day_count_basis_type == DayCountBasisTypes.THIRTY_E_360_ISDA:
            if start_day == 31: start_day = 30
            if end_day == 31: end_day = 30
            if start_month == 2 and is_eom(coupon_period_start) is True: start_day = 30
            if end_month == 2 and is_eom(settlement_date) is True and termination_date_flag is False: end_day = 30
            numerator = 360 * (end_year - start_year) + 30 * (end_month - start_month) + (end_day - start_day)
            denominator = 360
            acc_factor = numerator / denominator
            return acc_factor, numerator, denominator
        
        elif day_count_basis_type == DayCountBasisTypes.THIRTY_E_PLUS_360:
            if start_day == 31: start_day = 30
            if end_day == 31: end_month, end_day = end_month + 1, 1 
            numerator = 360 * (end_year - start_year) + 30 * (end_month - start_month) + (end_day - start_day)
            denominator = 360
            acc_factor = numerator / denominator
            return acc_factor, numerator, denominator
        
        elif day_count_basis_type == DayCountBasisTypes.ACT_ACT_ISDA or day_count_basis_type == DayCountBasisTypes.ZERO:
            if is_leap_year(coupon_period_start): strat_denominator = 366
            else: start_denominator = 365
            if is_leap_year(settlement_date): end_denominator = 366
            else: end_denominator = 365
            
            if start_year == end_year:
                numerator = (settlement_date - coupon_period_start).days
                denominator = start_denominator
                acc_factor = numerator / denominator
                return acc_factor, numerator, denominator
            
            else:
                start_days_year = (coupon_period_start - dt.date(start_year, 1, 1)).days
                end_days_year = (dt.date(end_year, 1, 1) - settlement_date).days
                acc_factor_start = start_days_year / start_denominator
                acc_factor_end = end_days_year / end_denominator
                numerator = start_days_year + end_days_year
                denominator = start_denominator + end_denominator
                year_diff = end_year - start_year - 1
                acc_factor = acc_factor_start + acc_factor_end + year_diff
                return acc_factor, numerator, denominator
            
        elif day_count_basis_type == DayCountBasisTypes.ACT_ACT_ICMA:
            frequency = get_annual_frequency(frequency_type)
            numerator = (settlement_date - coupon_period_start).days
            denominator = frequency * (coupon_period_end - coupon_period_start).days
            acc_factor = numerator / denominator
            return acc_factor, numerator, denominator
        
        elif day_count_basis_type == DayCountBasisTypes.ACT_365F:
            numerator = (settlement_date - coupon_period_start).days
            denominator = 365
            acc_factor = numerator / denominator
            return acc_factor, numerator, denominator
        
        elif day_count_basis_type == DayCountBasisTypes.ACT_360:
            numerator = (settlement_date - coupon_period_start).days
            denominator = 360
            acc_factor = numerator / denominator
            return acc_factor, numerator, denominator
        
        elif day_count_basis_type == DayCountBasisTypes.ACT_365L:
            frequency = get_annual_frequency(frequency_type)
            if coupon_period_end == None: year_end_coupon = end_year
            else: year_end_coupon = get_dmy(coupon_period_end)[2]
            if is_leap_year(coupon_period_start): feb29 = dt.date(start_year, 2, 29)
            elif is_leap_year(coupon_period_end): feb29 = dt.date(year_end_coupon, 2, 29)
            else: feb29 = dt.date(1900, 1, 1)
            if frequency == 1 and feb29 > coupon_period_start and feb29 <= coupon_period_end: denominator = 366
            elif is_leap_year(coupon_period_end): denominator = 366
            acc_factor = numerator / denominator
            return acc_factor, numerator, denominator
            
        elif day_count_basis_type == DayCountBasisTypes.SIMPLE:
            numerator = (settlement_date - coupon_period_start).days
            denominator = DaysInYear
            acc_factor = numerator / denominator
            return acc_factor, numerator, denominator
        
        else: 
            raise ValueError("The {} day_count_basis_type has not yet been implemented.".format(day_count_basis_type))
            
            
    def __repr__(self):
        return type(self).__name__
            

In [11]:
class CalendarTypes(Enum):
    NONE = 1
    WEEKEND = 2
    AUSTRALIA = 3
    CANADA = 4
    FRANCE = 5
    GERMANY = 6
    ITALY = 7
    JAPAN = 8
    NEW_ZEALAND = 9
    NORWAY = 10
    SWEDEN = 11
    SWITZERLAND = 12
    TARGET = 13
    UNITED_STATES = 14
    UNITED_KINGDOM = 15

In [12]:
class BusinessDayAdjustTypes(Enum):
    NONE = 1
    FOLLOWING = 2
    MODIFIED_FOLLOWING = 3
    PRECEDING = 4
    MODIFIED_PRECEDING = 5

In [13]:
class DateGenRuleTypes(Enum):
    FORWARD = 1
    BACKWARD = 2

In [94]:
# TODO: Exclude the relevant holidays from the below lists
# NOTE: The only case this Enum class works so far is for US
def is_business_day(date: dt.date, calendar_type: CalendarTypes):
    """Checks if a date is a work day given a calendar_type."""
    if calendar_type == CalendarTypes.NONE: return True
    elif date.weekday() == 5 or date.weekday() == 6: return False
    year = date.year
    holidays_dict = {CalendarTypes.AUSTRALIA: holidays.Australia(years = year),
                     CalendarTypes.CANADA: holidays.Canada(years = year),
                     CalendarTypes.FRANCE: holidays.France(years = year),
                     CalendarTypes.GERMANY: holidays.Germany(years = year),
                     CalendarTypes.ITALY: holidays.Italy(years = year),
                     CalendarTypes.JAPAN: holidays.Japan(years = year),
                     CalendarTypes.NEW_ZEALAND: holidays.NewZealand(years = year),
                     CalendarTypes.NORWAY: holidays.Norway(years = year),
                     CalendarTypes.SWEDEN: holidays.Sweden(years = year),
                     CalendarTypes.SWITZERLAND: holidays.Switzerland(years = year),
                     CalendarTypes.TARGET: {},
                     CalendarTypes.UNITED_STATES: holidays.XNYS(years = year),
                     CalendarTypes.UNITED_KINGDOM: holidays.UnitedKingdom(years = year)}
    return date not in holidays_dict[calendar_type].keys()

In [15]:
class Calendar(object):
    
    def __init__(self, calendar_type: CalendarTypes):
        
        self.calendar_type = calendar_type
      
    
    def adjust_date(self,
                    date: dt.date,
                    business_day_adjusment_type: BusinessDayAdjustTypes):
        
        calendar_type = self.calendar_type
        
        if calendar_type == CalendarTypes.NONE:
            return date
        
        elif business_day_adjusment_type == BusinessDayAdjustTypes.NONE:
            return date
        
        elif business_day_adjusment_type == BusinessDayAdjustTypes.FOLLOWING:
            while is_business_day(date, calendar_type) is False: date += dt.timedelta(days = 1)
            return date
            
        elif business_day_adjusment_type == BusinessDayAdjustTypes.MODIFIED_FOLLOWING:
            initial_date = date
            while is_business_day(date, calendar_type) is False: date += dt.timedelta(days = 1)
            if date.month != initial_date.month:
                while is_business_day(date, calendar_type) is False: date += dt.timedelta(days = -1)
            return initial_date
        
        elif business_day_adjusment_type == BusinessDayAdjustTypes.PRECEDING:
            while is_business_day(date, calendar_type) is False: date += dt.timedelta(days = -1)
            return date
        
        elif business_day_adjusment_type == BusinessDayAdjustTypes.MODIFIED_PRECEDING:
            initial_date = date
            while is_business_day(date, calendar_type) is False: date += dt.timedelta(days = -1)
            if date.month != initial_date.month:
                while is_business_day(date, calendar_type) is False: date += dt.timedelta(days = +1)
            return initial_date
        
        else:
            raise ValueError("The {}  business_day_adjusment_type has not yet been implemented.".format(business_day_adjusment_type))
        
    
    def add_business_days(self,
                         date: dt.date,
                         n_days: int,
                         calendar_type: CalendarTypes):
    
        count, new_date = date, 0    
        while count != n_days:
            new_date += dt.timedelta(days = 1)
            if is_business_day(new_date, calendar_type): count +=1
        return new_date
    
    
    def subtract_business_days(self,
                               n_days: int,
                               calendar_type: CalendarTypes):
    
        count, new_date = date, 0    
        while count != n_days:
            new_date += dt.timedelta(days = -1)
            if is_business_day(new_date, calendar_type): count +=1
        return new_date
        
    
    def __repr__(self):
        return type(self).__name__
    

In [93]:
class PaymentSchedule(object):
    
    def __init__(self,
                 effective_date: dt.date, # start date
                 termination_date: dt.date,
                 frequency_type: FrequencyTypes,
                 calendar_type: CalendarTypes,
                 business_day_adjusment_type: BusinessDayAdjustTypes,
                 date_gen_rule_type: DateGenRuleTypes,
                 adjust_termination_date: bool = True,
                 eom: bool = False, # All dates are eom
                 first_coupon_date: dt.date = None,
                 penultimate_coupon_date = None):
        
        self.effective_date = effective_date
        self.termination_date = termination_date
        
        if first_coupon_date == None: self.first_coupon_date = effective_date
        else: self.first_coupon_date = first_coupon_date # TODO: calc should be automatic
        
        if penultimate_coupon_date == None: self.penultimate_coupon_date = termination_date
        else: self.penultimate_coupon_date = penultimate_coupon_date # TODO: calc should be automatic
        
        self.frequency_type = frequency_type
        self.calendar_type = calendar_type
        self.business_day_adjusment_type = business_day_adjusment_type
        self.date_gen_rule_type = date_gen_rule_type
        self.adjust_termination_date = adjust_termination_date
        self.eom = eom
        
        self.adjusted_schedule_dates = []
        self.generate_payment_schedule()
        
        
    # TODO: Vectorize this fuction - current nested loop implementation is not efficient.
    # TODO: Figure out if the function is handling correctly effective_date and termination_date  
    def generate_payment_schedule(self):

        calendar_obj = Calendar(self.calendar_type)
        frequency = get_annual_frequency(self.frequency_type)
        num_months = int(12/frequency)
        unadjusted_schedule_dates = []

        if self.date_gen_rule_type == DateGenRuleTypes.BACKWARD:
            next_date = self.termination_date
            flow_num = 0

            while next_date > self.effective_date:
                unadjusted_schedule_dates.append(next_date)
                next_date = self.termination_date + relativedelta(months=-num_months*(1+flow_num))
                if self.eom and next_date > self.effective_date: 
                    next_date = dt.date(next_date.year,
                                        next_date.month,
                                        calendar.monthrange(next_date.year, next_date.month)[1])
                flow_num += 1
            
            unadjusted_schedule_dates.append(self.effective_date)
            flow_num += 1
            date = unadjusted_schedule_dates[flow_num - 1]
            self.adjusted_schedule_dates.append(date)

            for i in range(1, flow_num - 1):
                date = calendar_obj.adjust_date(unadjusted_schedule_dates[flow_num-i-1],
                                              self.business_day_adjusment_type)
                self.adjusted_schedule_dates.append(date)

            self.adjusted_schedule_dates.append(self.termination_date)

        # FIXME: This part is more than likely wrong
        elif self.date_gen_rule_type == DateGenRuleTypes.FORWARD:
            next_date = self.effective_date
            flow_num = 0

            while next_date < self.termination_date:
                unadjusted_schedule_dates.append(next_date)
                next_date = self.termination_date + relativedelta(months=num_months*(1+flow_num))
                if self.eom and next_date < self.termination_date:
                    next_date = dt.date(next_date.year, next_date.month, calendar.monthrange(next_date.year, next_date.month)[1])
                flow_num += 1

            self.adjusted_schedule_dates.append(effective_date)

            for i in range(1, flow_num-1):
                date = calendar_obj.adjust_date(unadjusted_schedule_dates[i],
                                                self.business_day_adjusment_type)
                self.adjusted_schedule_dates.append(date)

            if self.adjust_termination_date: 
                self.termination_date = calendar_obj.adjust_date(self.termination_date,
                                                                 self.business_day_adjusment_type)
            self.adjusted_schedule_dates.append(self.termination_date)


    def __repr__(self):
        return type(self).__name__
    
        

In [84]:
payment_schedule = PaymentSchedule(effective_date = dt.date(2023, 1, 10),
                                   termination_date = dt.date(2028, 1, 1),
                                   frequency_type = FrequencyTypes.QUARTERLY,
                                   calendar_type = CalendarTypes.UNITED_STATES,
                                   business_day_adjusment_type = BusinessDayAdjustTypes.FOLLOWING,
                                   date_gen_rule_type = DateGenRuleTypes.BACKWARD,
                                   adjust_termination_date = False,
                                   eom = True,
                                   first_coupon_date = None,
                                   penultimate_coupon_date = None)

### Discount Curve

In [98]:
import numpy as np
import datetime as dt
from enum import Enum

In [100]:
class InterpolatorTypes(Enum):
    LINEAR = 1
    NELSON_SIGEL = 2
    CUBIC_SPLINE = 3

In [99]:
class DiscountCurvesTypes(Enum):
    LIBOR = 1
    SOFR = 2

In [101]:
class DiscountCurve(object):
    
    def __init__(self,
                 valuation_date: dt.date,
                 dates: list,
                 discount_factors: list,
                 calendar_type: CalendarTypes,
                 interpolator_type: InterpolatorTypes): 
            
        self.valuation_date = valuation_date
        self.dates = dates
        self.discount_factors = discount_factors
        self.calendar_type = calendar_type
        self.interpolator_type = interpolator_type
        self.jumps = jumps
        self.jump_dates = jump_dates
    
    
    
    

### Interest Rate Swap

In [96]:
import numpy as np
import datetime as dt
from enum import Enum 

In [95]:
class SwapTypes(Enum):
    PAY = 1
    RECEIVE = 2

In [ ]:
class IRSwapLeg(object):
    
    def __init__(self):
        
        pass
    
    
    def build_cash_flows(self):
        
        pass
        
        
        
    
    
    
    

### Interest Rate Swap

In [ ]:
class IRSwap(object):
    
    def __init__(self,
                 pay_or_receive,
                 notional,
                 termination_date,
                 effective_date,
                 frequency,
                 basis):
        
        self.pay_or_receive = pay_or_receive 
        self.notional = notional
        self.termination_date = termination_date
        self.effective_date = effective_date
        self.frequency = frequency
        self.basis = basis

### Interest Rate Swaption